In [1]:
import FI_estimation
import torch
from torch import nn
from torch.nn import functional as F
import torch.utils.data as Data
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch import optim
import numpy as np
from matplotlib import pyplot as plt
import os
from scipy import ndimage

torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor

/home/maximilian/anaconda3/envs/ML/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# regular experimental data
type = np.float64
x_tr = np.load("Data"+os.sep+"x_tr.npy")
y_tr = np.load("Data"+os.sep+"y_tr.npy")
x_tst = np.load("Data"+os.sep+"x_tst.npy")
delta = 0.3
#
# crop the positions
steps = 13
x_tr = x_tr.reshape(steps,50000,1,18,24)
y_tr = y_tr.reshape(steps,50000)
idx = np.arange(50000)
np.random.shuffle(idx)
x_tr = x_tr[:,idx]
y_tr = y_tr[:,idx]
#
y_tr = y_tr.reshape(y_tr.shape[0]*y_tr.shape[1])
x_tr = x_tr.reshape(x_tr.shape[0]*x_tr.shape[1],1,18,24)
print("training sample size: ", y_tr.shape[0])

training sample size:  650000


In [3]:
def shuffle_split_data(X, y, ratio=0.5):
    arr_rand = np.random.rand(X.shape[0])
    split = arr_rand < np.percentile(arr_rand, ratio*100)

    X_train = X[split]
    y_train = y[split]
    X_test =  X[~split]
    y_test = y[~split]
    return X_train, X_test, y_train, y_test

a = x_tr.shape[0]
std = np.std(x_tr, axis=0)
std[std==0] = 1
mean = np.mean(x_tr, axis=0)

for i in range(a): 
    x_tr[i] = (x_tr[i]-mean)/std
for i in range(x_tst.shape[0]): 
    x_tst[i] = (x_tst[i]-mean[0])/std[0]
# ###
# cf = 4 * (np.max(y_tr)/2)**2
# print("multiply the output variance with", cf)
# y_tr = y_tr-np.min(y_tr)
# y_tr = 2 * ( y_tr / np.max(y_tr) - 0.5 )

X_train, X_val, Y_train, Y_val = shuffle_split_data(x_tr, y_tr, ratio=0.8)
x_tr = None
y_tr = None

X_train = X_train.astype(type)
Y_train = Y_train.astype(type)
X_val = X_val.astype(type)
Y_val = Y_val.astype(type)
x_tst = x_tst.astype(np.float64)

In [4]:
torch.set_default_tensor_type(torch.DoubleTensor)
train_set = TensorDataset(torch.tensor(X_train), torch.tensor(Y_train).T)
val_set = TensorDataset(torch.tensor(X_val), torch.tensor(Y_val).T)

bs = 128
train_loader = Data.DataLoader(train_set, batch_size=bs, shuffle=True)
validation_loader = Data.DataLoader(val_set, batch_size=bs, shuffle=True)

/tmp/ipykernel_7061/1397782601.py:2: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755832681/work/aten/src/ATen/native/TensorShape.cpp:2318.)
  train_set = TensorDataset(torch.tensor(X_train), torch.tensor(Y_train).T)


In [5]:
class Model(nn.Module):
    def __init__(self, n):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(n[0], n[1])
        self.linear2 = nn.Linear(n[1], n[2])
        self.linear3 = nn.Linear(n[2], n[3])
        self.linear4 = nn.Linear(n[3], n[4])
        self.linear5 = nn.Linear(n[4], n[5])
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)    
        x = F.relu(x)
        x = self.linear3(x)
        x = F.relu(x)
        x = self.linear4(x)       
        x = F.relu(x)
        x = self.linear5(x)  
        return x
n=[18*24,800,100,25,5,1]

In [6]:
starting_epoch = 0
criterion =  nn.MSELoss()
n_epochs = 25
lr = 5e-7
training_loss = 0.
validation_loss = 0.
net = Model(n).to(device)
all_losses = []
all_losses_val = []
all_losses.append(training_loss/len(train_loader))
all_losses_val.append(validation_loss/len(validation_loader))
torch.save(net,  'Models' +os.sep+ 'epoch' + str(0)+'Experimental.pth')

for epoch in range(starting_epoch, starting_epoch + n_epochs):
    optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=0.003)
    net.train()
    training_loss = 0.
    validation_loss = 0.
    batch_count = 0
    for inputs, target in train_loader:
        inputs = inputs.to(device)
        target = target.to(device)
        batch_count += 1
        y_pred = net(inputs)
        loss = criterion(y_pred,  torch.unsqueeze(target,dim=1)  )
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
    net.eval()
    with torch.no_grad():
        for inputs, target in validation_loader:
            inputs = inputs.to(device)
            target = target.to(device)
            y_pred = net(inputs)
            loss_val = criterion(y_pred,  torch.unsqueeze(target,dim=1) )
            validation_loss += loss_val.item()
    print("epoch= ", epoch, ", val= ", validation_loss/len(validation_loader),", train = ", training_loss/len(train_loader))
    torch.save(net,   'Models' +os.sep+ 'epoch' + str(epoch+1)+'Experimental.pth')
    all_losses_val.append(validation_loss/len(validation_loader))
    all_losses.append(training_loss/len(train_loader))

epoch=  0 , val=  1.302305463744545 , train =  1.3362887694762988


KeyboardInterrupt: 

For FI based early stopping, increase the size of the network and decrease the learning rate
If y_tr is rescaled then the MSE must be rescaled accordingly in order to be comparable with 1/FI

In [28]:
# np.savez('Plots'+os.sep+'losses_SpaceInvader.npz',
#          all_losses=all_losses,
#         all_losses_val = all_losses_val)

In [6]:
# x_tst = x_tst.astype(np.float64)
# x_minus = x_tst[0]
# x_middle = x_tst[1]
# x_plus = x_tst[2]

In [ ]:
#------- calculating the FI flow at different epochs
#------- parameters entering the algorithm
fi_in = 1.6 # input FI should generally be evaluated carefully, possible by visual inspection of the LFI curve
kappa = 0.1 # ratio of slopes
gamma = 0.05 # stop when the LFI doesn't improve in the first iterations 
noise = 0.01 # noise added
step = 30 # increase of dimensionality
upperLimit = 200 # maximum increase 
n_epochs = 20
X = x_tst
#X = X.reshape(X.shape[0]*X.shape[1], X.shape[2]) 
X = torch.tensor(X)
print("X.shape", X.shape)
fi_array = []
for epoch in range(n_epochs):
    net = Model(n)
    net = torch.load('Models'+os.sep + 'epoch'+ str(epoch)+'Experimental.pth', map_location=torch.device('cpu'))
    net.eval()
    #------- extract data arrays
    return_nodes = {
        "linear1": "linear1",
        "linear2": "linear2",
        "linear3": "linear3",
        "linear4": "linear4"}
    model2 = create_feature_extractor(net, return_nodes=return_nodes)
    intermediate_outputs = model2(X)
    linear1 = intermediate_outputs['linear1']
    linear2 = intermediate_outputs['linear2']
    linear3 = intermediate_outputs['linear3']
    linear4 = intermediate_outputs['linear4']
    layer_out = net(X)
    #------- reshape arrays
    linear1 = linear1.reshape(3 , linear1.shape[0]//3, linear1.shape[1])
    linear2 = linear2.reshape(3 , linear2.shape[0]//3, linear2.shape[1])
    linear3 = linear3.reshape(3 , linear3.shape[0]//3, linear3.shape[1])
    linear4 = linear4.reshape(3 , linear4.shape[0]//3, linear4.shape[1])
    layer_out = layer_out.reshape(3 , layer_out.shape[0]//3, layer_out.shape[1])
    #------- convert to numpy arrays
    linear1 = linear1.detach().numpy()
    linear2 = linear2.detach().numpy()
    linear3 = linear3.detach().numpy()
    linear4 = linear4.detach().numpy()
    layer_out = layer_out.detach().numpy()
    #------- apply activation function since the data is extracted directly after the linear transformation
    linear1 = (FI_estimation.ReLU(linear1))
    linear2 = (FI_estimation.ReLU(linear2))
    linear3 = (FI_estimation.ReLU(linear3))
    linear4 = (FI_estimation.ReLU(linear4))
    layer_out = (layer_out)
    #------- calculate the FI
    fi_1 = FI_estimation.get_FI(linear1, delta, upperLimit, step, kappa, constant_threshold=gamma, noise_factor=noise, biasedLFI=True)
    fi_2 = FI_estimation.get_FI(linear2, delta, upperLimit, step, kappa, constant_threshold=gamma, noise_factor=noise, biasedLFI=True)
    fi_3 = FI_estimation.get_FI(linear3, delta, upperLimit, step, kappa, constant_threshold=gamma, noise_factor=noise, biasedLFI=True)
    fi_4 = FI_estimation.get_FI(linear4, delta, upperLimit, step, kappa, constant_threshold=gamma, noise_factor=noise, biasedLFI=True)
    fi_out = FI_estimation.get_FI(layer_out, delta, upperLimit, step, kappa, constant_threshold=gamma, noise_factor=noise, biasedLFI=True)

    if isinstance(fi_out, np.ndarray): fi_out = fi_out.item()
    fi = np.array([fi_in,fi_1,fi_2,fi_3,fi_4,fi_out])
    print(fi)
    fi_array.append(fi)
    print("#------------------------------------------------------#")

X.shape torch.Size([450000, 18, 24])
calculating FI
up:  0
up:  30
up:  60
LFI didn't increase after the 3rd iteration:  860
calculating FI
up:  0
up:  30
up:  60
LFI didn't increase after the 3rd iteration:  160
calculating FI
up:  0
up:  30
up:  60
up:  90
up:  120
up:  150
up:  180
calculating FI
up:  0
up:  30
up:  60
up:  90
up:  120
up:  150
up:  180
calculating FI
up:  0
up:  30
up:  60
up:  90
up:  120
up:  150
up:  180
[1.60000000e+00 1.16838807e+00 1.72955909e-01 2.78382529e-02
 1.47825281e-03 1.78705140e-03]
#------------------------------------------------------#
calculating FI
up:  0


In [8]:
#np.save("Plots" + os.sep + "fi_flow_SpaceInvader.npy", np.array(fi_array))